In [1]:
import numpy as np
#!sudo pip3 install --upgrade --force-reinstall scipy

In [8]:
#!sudo pip install PyWavelets
#!sudo pip3 install --upgrade --force-reinstall numpy
#import numpy as np
from PIL import Image
from Helper_functions import *
# Import functions and libraries
import numpy as np
import matplotlib.pyplot as plt
import queue as Queue
import time
import sys
import base64


from numpy import pi
from numpy import sin
from numpy import zeros
from numpy import r_
from numpy import ones

import threading
from bitarray import bitarray as tempbit

from numpy import mean
from numpy import power
from numpy.fft import fft
from numpy.fft import fftshift
from numpy.fft import ifft
from numpy.fft import ifftshift
import bitarray

import newax25 as ax25
import pywt
import pywt.data
from os import stat
from PIL import Image


import multiprocessing
from huffman import *

from math import gcd
import sounddevice as sd
import RPi.GPIO as GPIO
from functools import reduce
from numpy import ones,zeros, pi, cos, exp, sign


%matplotlib inline


In [9]:
from scipy import integrate
from  scipy.io.wavfile import read as wavread
from scipy import signal
from scipy.fftpack import dct, idct

In [10]:
!sudo modprobe snd_aloop
!aplay -l | grep -i loopback

card 2: Loopback [Loopback], device 0: Loopback PCM [Loopback PCM]
card 2: Loopback [Loopback], device 1: Loopback PCM [Loopback PCM]


In [11]:
sd.query_devices()

  0 bcm2835 ALSA: - (hw:0,0), ALSA (0 in, 2 out)
  1 bcm2835 ALSA: IEC958/HDMI (hw:0,1), ALSA (0 in, 2 out)
  2 Fe-Pi Audio: - (hw:1,0), ALSA (2 in, 2 out)
  3 Loopback: PCM (hw:2,0), ALSA (32 in, 32 out)
  4 Loopback: PCM (hw:2,1), ALSA (32 in, 32 out)
  5 sysdefault, ALSA (0 in, 128 out)
  6 dmix, ALSA (0 in, 2 out)
* 7 default, ALSA (2 in, 2 out)

In [12]:
builtin_idx = 0
usb_idx = 2
loop_idx = 4  # Use loopback dev 1! direwolf is connected to dev 0.
loop_idx_rcv = 3 # Use loopback dev 0. 
sd.default.samplerate=48000
sd.default.channels = 1

In [13]:
# function to compute least common multipler
def lcm(numbers):
    return reduce(lambda x, y: (x*y)//gcd(x,y), numbers, 1)


In [14]:
import numpy.ctypeslib as npct
from ctypes import c_int
from ctypes import c_float

array_1d_int = npct.ndpointer(dtype=np.int, ndim=1, flags='CONTIGUOUS')

libcd = npct.load_library("./libpll", ".")
libcd.pll.restype = c_int
libcd.pll.argtypes= [array_1d_int, c_int, array_1d_int,array_1d_int,  array_1d_int,array_1d_int, c_int, c_float]


class TNCaprs: 
    def __init__(self, fs = 48000.0, Abuffer = 1024, Nchunks=10):
        
        #  Implementation of an afsk1200 TNC. 
        #
        #  The TNC processes a `Abuffer` long buffers, till `Nchunks` number of buffers are 
        #  collected into a large one.
        #  This is because python is able to more efficiently process larger buffers than smaller ones.
        #  Then, the resulting large buffer is demodulated, sampled and packets extracted.
        #
        # Inputs:
        #    fs  - sampling rate
        #   TBW  -  TBW of the demodulator filters
        #   Abuffer - Input audio buffers from Pyaudio
        #   Nchunks - Number of audio buffers to collect before processing
        #   plla    - agressivness parameter of the PLL
        
        ## compute sizes based on inputs
        self.TBW = 2.0   # TBW for the demod filters
        self.N = (int(fs/1200*self.TBW)//2)*2+1   # length of the mark-space filters for demod
        self.fs = fs     # sampling rate   
        self.BW = 1200      # BW of filter based on TBW
        self.Abuffer = Abuffer             # size of audio buffer
        self.Nchunks = Nchunks             # number of audio buffers to collect
        self.Nbuffer = Abuffer*Nchunks+(self.N*3-3)         # length of the large buffer for processing
        self.Ns = 1.0*fs/1200.0 # samples per symbol
        
        ## state variables for the modulator
        self.prev_ph = 0  # previous phase to maintain continuous phase when recalling the function
        
        ##  Generate Filters for the demodulator
        self.h_lp = signal.firwin(self.N,self.BW/self.fs*1.0,window='hanning')#TODO
        self.h_lpp = signal.firwin(self.N,self.BW*2*1.2/self.fs,window='hanning')#TODO
        self.h_space = self.h_lp*exp(1j*2*pi*(2200)*r_[-self.N/2:self.N/2]/self.fs)#TODO
        self.h_mark = self.h_lp*exp(1j*2*pi*(1200)*r_[-self.N/2:self.N/2]/self.fs)#TODO
        self.h_bp = (signal.firwin(self.N,self.BW/self.fs*2.2,window='hanning'))*exp(1j*2*pi*1700*r_[-self.N/2:self.N/2]/self.fs)#TODO

        ## PLL state variables  -- so conntinuity between buffers is preserved
        self.dpll = np.round(2.0**32 / self.Ns).astype(np.int32)    # PLL step
        self.pll =  0                # PLL counter
        self.ppll = -self.dpll       # PLL counter previous value -- to detect overflow
        self.plla = 0.74             # PLL agressivness (small more agressive)
    
        ## state variable to NRZI2NRZ
        self.NRZIprevBit = bool(1)  
        
        ## State variables for findPackets
        self.state='search'   # state variable:  'search' or 'pkt'
        self.pktcounter = 0   # counts the length of a packet
        self.packet = bitarray.bitarray([0,1,1,1,1,1,1,0])   # current packet being collected
        self.bitpointer = 0   # poiter to advance the search beyond what was already 
                              # searched in the previous buffer

        ## State variables for processBuffer
        self.buff = zeros(self.Nbuffer)   # large overlapp-save buffer
        self.chunk_count = 0              # chunk counter
        # bits from end of prev buffer to be copied to beginning of new
        self.oldbits = bitarray.bitarray([0,0,0,0,0,0,0])    
        self.Npackets = 0                 # packet counter
        
    
    def NRZ2NRZI(self,NRZ, prevBit = True):
        NRZI = NRZ.copy() 
        for n in range(0,len(NRZ)):
            if NRZ[n] :
                NRZI[n] = prevBit
            else:
                NRZI[n] = not(prevBit)
            prevBit = NRZI[n]
        return NRZI
    

    def NRZI2NRZ(self, NRZI):  
        NRZ = NRZI.copy() 
    
        for n in range(0,len(NRZI)):
            NRZ[n] = NRZI[n] == self.NRZIprevBit
            self.NRZIprevBit = NRZI[n]
    
        return NRZ
    
    
    def KISS2bits(self,KISS):
        # function that takes a KISS frame sent via TCP/IP and converts it to an APRSpacket bit stream.
        
        bits = bitarray.bitarray(endian="little")
        bits.frombytes(KISS)
        fcs = ax25.FCS()
        for bit in bits:
            fcs.update_bit(bit)
            
        bits.frombytes(fcs.digest())
        return bitarray.bitarray('01111110') + ax25.bit_stuff(bits) + bitarray.bitarray('01111110') 
     
        
    def bits2KISS(self,bits):
        # function that takes a bitstream of an APRS-packet, removes flags and FCS and unstuffs the bits
        bitsu = ax25.bit_unstuff(bits[8:-8])
        return  bitsu[:-16].tobytes() 
    
    
    def modulate(self,bits):
    # the function will take a bitarray of bits and will output an AFSK1200 modulated signal of them, 
    # sampled at fs Hz
    #  Inputs:
    #         bits  - bitarray of bits
    #         fs    - sampling rate
    # Outputs:
    #         sig    -  returns afsk1200 modulated signal
        # For you to complete  
        fss = lcm((1200,self.fs))
        deci = fss//fs

        Nb = fss//1200
        nb = len(bits)
        NRZ = ones((nb,Nb))
        for n in range(0,nb):
            if bits[n]:
                NRZ[n,:]=-NRZ[n,:]

        freq = 1700 + 500*NRZ.ravel()
        ph = 2.0*pi*integrate.cumtrapz(freq)/fss
        sig = cos(ph[::deci])

        return sig 
    
    
    def modulatePacket(self, callsign, digi, dest, info, preflags=80, postflags=80 ):
        
        # given callsign, digipath, dest, info, number of pre-flags and post-flags the function contructs
        # an appropriate aprs packet, then converts them to NRZI and calls `modulate` 
        # to afsk 1200 modulate the packet. 
        
        packet = ax25.UI(destination=dest,source=callsign, info=info, digipeaters=digi.split(b','),)
        prefix = bitarray.bitarray(np.tile([0,1,1,1,1,1,1,0],(preflags,)).tolist())
        suffix = bitarray.bitarray(np.tile([0,1,1,1,1,1,1,0],(postflags,)).tolist())
        sig = self.modulate(self.NRZ2NRZI(prefix + packet.unparse()+suffix))

        return sig
    

    def demod(self, buff):
        #Demodulates a buffer and returns valid NRZ
    
        # Similar to afsk1200_demod,  for you to complete

        sig = np.convolve(buff.copy(),self.h_bp,mode='valid')
        mark = abs(np.convolve(sig,self.h_mark,mode='valid'))
        space = abs(np.convolve(sig,self.h_space,mode='valid'))
        NRZ = mark-space
        NRZ = np.convolve(NRZ,self.h_lpp,'valid')

        
        return NRZ


    def FastPLL(self,NRZa):
        recbits = np.zeros(len(NRZa)//(self.fs//1200)*2,dtype=np.int32)
        pll = np.zeros(1,dtype = np.int32)
        pll[0] = self.pll
        ppll = np.zeros(1,dtype = np.int32)
        ppll[0] = self.ppll
        
        #print("pll = ",pll,"   ppll=",ppll)
        
        NRZb = (NRZa > 0).astype(np.int32)
        tot = libcd.pll(NRZb,len(NRZb),recbits,recbits,pll,ppll,self.dpll,self.plla)
        
        self.ppll = ppll.copy()
        self.pll = pll.copy()
        
        #print("post: pll = ",pll,"   ppll=",ppll)
        
        return bitarray.bitarray(recbits[:tot].tolist())
    
    def PLL(self, NRZa):
       #print("running PLL")
        idx = zeros(len(NRZa)//int(self.Ns)*2)   # allocate space to save indexes        
        c = 0
        
        for n in range(1,len(NRZa)):
            if (self.pll < 0) and (self.ppll >0):
                idx[c] = n
                c = c+1
        
            if (NRZa[n] >= 0) !=  (NRZa[n-1] >=0):
                self.pll = np.int32(self.pll*self.plla)    
        
            self.ppll = self.pll
            self.pll = np.int32(self.pll+ self.dpll)
    
        return idx[:c].astype(np.int32) 
    

    def findPackets(self,bits):
        # function take a bitarray and looks for AX.25 packets in it. 
        # It implements a 2-state machine of searching for flag or collecting packets
        flg = bitarray.bitarray([0,1,1,1,1,1,1,0])
        packets = []
        n = self.bitpointer
        
        # Loop over bits
        while (n < len(bits)-7) :
            # default state is searching for packets
            if self.state is 'search':
                # look for 1111110, because can't be sure if the first zero is decoded
                # well if the packet is not padded.
                if bits[n:n+7] == flg[1:]:
                    # flag detected, so switch state to collecting bits in a packet
                    # start by copying the flag to the packet
                    # start counter to count the number of bits in the packet
                    self.state = 'pkt'
                    self.packet=flg.copy()
                    self.pktcounter = 8
                    # Advance to the end of the flag
                    n = n + 7
                else:
                    # flag was not found, advance by 1
                    n = n + 1             
            # state is to collect packet data. 
            elif self.state is 'pkt':
                # Check if we reached a flag by comparing with 0111111
                # 6 times ones is not allowed in a packet, hence it must be a flag (if there's no error)
                if bits[n:n+7] == flg[:7]:
                    # Flag detected, check if packet is longer than some minimum
                    if self.pktcounter > 200:
                        #print('packet found!')
                        # End of packet reached! append packet to list and switch to searching state
                        # We don't advance pointer since this our packet might have been
                        # flase detection and this flag could be the beginning of a real packet
                        self.state = 'search'
                        self.packet.extend(flg)
                        packets.append(self.packet.copy())
                    else:
                        # packet is too short! false alarm. Keep searching 
                        # We don't advance pointer since this this flag could be the beginning of a real packet
                        self.state = 'search'
                # No flag, so collect the bit and add to the packet
                else:
                    # check if packet is too long... if so, must be false alarm
                    if self.pktcounter < 2680:
                        # Not a false alarm, collect the bit and advance pointer        
                        self.packet.append(bits[n])
                        self.pktcounter = self.pktcounter + 1
                        n = n + 1
                    else:  #runaway packet
                        #runaway packet, switch state to searching, and advance pointer
                        self.state = 'search'
                        n = n + 1
        
        self.bitpointer = n-(len(bits)-7) 
        return packets

    
    # function to generate a checksum for validating packets
    def genfcs(self,bits):
        # Generates a checksum from packet bits
        fcs = ax25.FCS()
        for bit in bits:
            fcs.update_bit(bit)
    
        digest = bitarray.bitarray(endian="little")
        digest.frombytes(fcs.digest())

        return digest


    # function to parse packet bits to information
    def decodeAX25(self, bits):
        ax = ax25.AX25()
        ax.info = "bad packet"
        ax.parse(bits)
        return ax
    
    
    def processBuffer(self, buff_in):
        
        # function processes an audio buffer. It collect several small into a large one
        # Then it demodulates and finds packets.
        #
        # The function uses overlap and save convolution
        # The function returns packets when they become available. Otherwise, returns empty list
        
        N = self.N
        NN = (N*3 -3 )
        
        Nchunks = self.Nchunks
        Abuffer = self.Abuffer
        fs = self.fs
        Ns = self.Ns
        
        validPackets=[]
        packets=[]
        NRZI=[]
        idx = []
        bits = []
        
        # Fill in buffer at the right place
        self.buff[NN+self.chunk_count*Abuffer:NN+(self.chunk_count+1)*Abuffer] = buff_in.copy()
        self.chunk_count = self.chunk_count + 1  
        
        # number of chunk reached -- process large buffer
        if self.chunk_count == Nchunks:
            # Demodulate to get NRZI
            NRZI = self.demod(self.buff)
            # compute sampling points, using PLL
            #idx = self.PLL(NRZI)
            # Sample and make a decision based on threshold
            #bits = bitarray.bitarray((NRZI[idx]>0).tolist())
            
            bits = self.FastPLL(NRZI)
            # In case that buffer is too small raise an error -- must have at least 7 bits worth
            if len(bits) < 7:
                raise ValueError('number of bits too small for buffer')
            
            # concatenate end of previous buffer to current one
            bits = self.oldbits + self.NRZI2NRZ(bits)
            
            # store end of bit buffer to next buffer
            self.oldbits = bits[-7:].copy()
            
            # look for packets
            packets = self.findPackets(bits)
            
            # Copy end of sample buffer to the beginning of the next (overlapp and save)
            self.buff[:NN] = self.buff[-NN:].copy()
            
            # reset chunk counter
            self.chunk_count = 0
            
            # checksum test for all detected packets
            for n in range(0,len(packets)):
                if len(packets[n]) > 200: 
                    try:
                        ax = self.decodeAX25(packets[n])
                    except:
                        ax = ax25.AX25()
                        ax.info = "bad packet"
                    if ax.info != 'bad packet' and ax.info != 'no decode':
                        validPackets.append(packets[n])                
  
        return validPackets

In [24]:
# # Convert from bitarray class to b64 bytes and back
# def ee123_bitarr_to_base64(bits: bitarray.bitarray):
#     bN = np.uint32(len(bits)).tobytes()
#     Bytes = bN + bits.tobytes()
#     return base64.b64encode(Bytes)

# def ee123_base64_to_bitarr(b64: str):
#     Bytes = base64.b64decode(b64)  # Be careful not to overwrite the builtin bytes class!
#     N = np.frombuffer(Bytes[:4], dtype='<u4')[0]
#     ba = bitarray.bitarray()
#     ba.frombytes(Bytes[4:])
#     return ba[:N]
# def compressed_to_b64(fname, compression,rate): 
#     # compressed is a variable that contains the compressed data
#     with open(fname, 'rb') as f:
#         raw = f.read()
#     # raw = file we have compressed 
#     ret_ = compression(raw,rate=rate) # should this be .bin file?
#     b64 = base64.b64encode(ret_)
#     return b64


def file_to_b64(fname):
    with open(fname, 'rb') as f:
        raw = f.read()
    # raw = file we have compressed
    #Prepend the number of bytes before encoding
    print('len:', len(raw),"\ntobyes: ",np.uint32(len(raw)).tobytes(),type(raw))
    b64 = base64.b64encode(np.uint32(len(raw)).tobytes() + raw)
    return b64
def enqueue_data(callsign, modem, data, address=None, uid=None, fname="myfile.bin", 
                 comment="UCB EE123 message format", dest=b"APCAL", bsize=240):
    """
    Inputs:
      callsign: your callsign
      modem: a modem object
      data: data to send as a bytes object
      address: address callsign for the message, defaults to your callsign
      uid: optionally specify a UID, otherwise one will be randomly chosen
      fname: filename to send in start packet
      comment: comment for start packet
      dest: should be APCAL for EE123 data
      bsize: number of bytes to send per packet
      
    Outputs:
      Qout: a queue containing modulated packets to transmit
      uid: the uid used for this data transfer
    """
    if address is None:
        address = callsign
    if uid is None:
        uid = np.random.randint(0, 10000)
    uid = bytes("{:04d}".format(uid), 'utf-8')
    print("Putting packets in Queue for transmission ID=%s" % uid.decode())
    
    Qout = Queue.Queue()
    
    # For you to complete
     # For you to complete
    
    # Enqueue start transmission
    addr = bytes(':{:<9}:'.format(address), 'utf-8')
    start = b','.join([addr + b'START' + uid, bytes(fname, 'utf-8'), bytes(comment, 'utf-8')])
    print("start:", start)
    print(type(start))
    tmp = modem.modulatePacket(callsign, b"WIDE1-1", dest, start, preflags=80, postflags=10 ) 
    Qout.put(tmp)
    idx = 0
    while(1):
        seq = bytes("{:04d}".format(idx), 'utf-8')
        pkt = addr + data[idx*bsize:idx*bsize + bsize] + seq
        tmp = modem.modulatePacket(callsign, b"WIDE1-1", dest, pkt, preflags=20, postflags=10 )    
        Qout.put(tmp)
    
        idx = idx+1
        if idx * bsize > len(data):
            break
    end = addr + b'END' + uid
    print("end:", end)
    tmp = modem.modulatePacket(callsign, b"WIDE1-1", dest, end, preflags=2, postflags=80 )
    Qout.put(tmp)
    
    return Qout, uid

In [25]:
def RGB2YCbCr(im_rgb):
    # Input:  a 3D float array, im_rgb, representing an RGB image in range [0.0,255.0]
    # Output: a 3D float array, im_ycbcr, representing a YCbCr image in range [-128.0,127.0]
    
    # Your code here
    im_ycbcr = np.empty(im_rgb.shape, dtype=np.float64)
    im_ycbcr[:,:,0]  = 0.299*im_rgb[:,:,0] + 0.587*im_rgb[:,:,1] + 0.114*im_rgb[:,:,2] - 128.
    im_ycbcr[:,:,1] = -0.168736*im_rgb[:,:,0] - 0.331264*im_rgb[:,:,1] + 0.5*im_rgb[:,:,2]
    im_ycbcr[:,:,2] = 0.5*im_rgb[:,:,0] - 0.418688*im_rgb[:,:,1] - 0.081312*im_rgb[:,:,2]
    
    return im_ycbcr

def chroma_downsample(C):
    # Input:  an MxN array, C, of chroma values
    # Output: an (M/2)x(N/2) array, C2, of downsampled chroma values
    
    # Your code here
    M, N = C.shape
    C2 = np.array(Image.fromarray(C).resize((N//2,M//2), resample=Image.BILINEAR))
    
    return C2

def quantize(block_c, mode="y", quality=75):
    # Input:  a 2D float array, block_c, of DCT coefficients
    #         a string, mode, ("y" for luma quantization, "c" for chroma quantization)
    #         an int, quality, the JPEG quality factor in range [1,100] (defaults to 75)
    # Output: a 2D int array, block_cq, of quantized DCT coefficients
    
    if mode is "y":
        Q = np.array([[ 16,  11,  10,  16,  24,  40,  51,  61 ],
                      [ 12,  12,  14,  19,  26,  58,  60,  55 ],
                      [ 14,  13,  16,  24,  40,  57,  69,  56 ],
                      [ 14,  17,  22,  29,  51,  87,  80,  62 ],
                      [ 18,  22,  37,  56,  68,  109, 103, 77 ],
                      [ 24,  36,  55,  64,  81,  104, 113, 92 ],
                      [ 49,  64,  78,  87,  103, 121, 120, 101],
                      [ 72,  92,  95,  98,  112, 100, 103, 99 ]])
    elif mode is "c":
        Q = np.array([[ 17,  18,  24,  47,  99,  99,  99,  99 ],
                      [ 18,  21,  26,  66,  99,  99,  99,  99 ],
                      [ 24,  26,  56,  99,  99,  99,  99,  99 ],
                      [ 47,  66,  99,  99,  99,  99,  99,  99 ],
                      [ 99,  99,  99,  99,  99,  99,  99,  99 ],
                      [ 99,  99,  99,  99,  99,  99,  99,  99 ],
                      [ 99,  99,  99,  99,  99,  99,  99,  99 ],
                      [ 99,  99,  99,  99,  99,  99,  99,  99 ]]) 
    else:
        raise Exception("String argument must be 'y' or 'c'.")
        
    if quality < 1 or quality > 100:
        raise Exception("Quality factor must be in range [1,100].")
    
    scalar = 5000 / quality if quality < 50 else 200 - 2 * quality # formula for scaling by quality factor
    Q = Q * scalar / 100. # scale the quantization matrix
    Q[Q<1.] = 1. # do not divide by numbers less than 1
    
    # Your code here
    block_cq = (block_c / Q).round().astype(int)
    
    return block_cq

def zigzag(block_cq):
    # Input:  a 2D array, block_cq, of quantized DCT coefficients
    # Output: a list, block_cqz, of zig-zag reordered quantized DCT coefficients
    
    idx = [0, 1, 8, 16, 9, 2, 3, 10, 17, 24, 32, 25, 18, 11, 4, 5, 12, 19, 26, 33, 40, 48, 41,
           34, 27, 20, 13, 6, 7, 14, 21, 28, 35, 42, 49, 56, 57, 50, 43, 36, 29, 22, 15, 23,
           30, 37, 44, 51, 58, 59, 52, 45, 38, 31, 39, 46, 53, 60, 61, 54, 47, 55, 62, 63]
    
    # Your code here
    block_cqz = block_cq.ravel()[idx].tolist()
    
    return block_cqz

def zrle(block_cqz):
    # Input:  a list, block_cqz, of zig-zag reordered quantized DCT coefficients
    # Output: a list, block_cqzr, of zero-run-length encoded quantized DCT coefficients
    
    # Your code here
    block_cqzr = [block_cqz[0]] # initialize list with DC value
    ind = 1
    while np.count_nonzero(block_cqz[ind:]) > 0:
        run = np.nonzero(block_cqz[ind:])[0][0] # run length
        if run > 15:
            block_cqzr.append((15,0))
            ind += 16
            continue
        val = block_cqz[ind:][run] # nonzero value
        block_cqzr.append((run,val)) # append tuple
        ind += run + 1
    block_cqzr.append((0,0)) # terminate list with (0,0)
    
    return block_cqzr
def dct2(block):
    # Input:  a 2D array, block, representing an image block
    # Output: a 2D array, block_c, of DCT coefficients
    
    # Your code here
    block_c = dct(block, type=2, norm='ortho', axis=0)
    block_c = dct(block_c, type=2, norm='ortho', axis=1)
    
    return block_c

def encode_block(block, mode="y", quality=75):
    # Input:  a 2D array, block, representing an image component block
    #         a string, mode, ("y" for luma, "c" for chroma)
    #         an int, quality, the JPEG quality factor in range [1,100] (defaults to 75)
    # Output: a bitarray, dc_bits, of Huffman encoded DC coefficients
    #         a bitarray, ac_bits, of Huffman encoded AC coefficients
    block_c = dct2(block)
    block_cq = quantize(block_c, mode, quality)
    block_cqz = zigzag(block_cq)
    block_cqzr = zrle(block_cqz)
    dc_bits = encode_huffman(block_cqzr[0], mode) # DC
    ac_bits = ''.join(encode_huffman(v, mode) for v in block_cqzr[1:]) # AC
    return bitarray.bitarray(dc_bits), bitarray.bitarray(ac_bits)
def mirror_pad(img):
    # Input:  a 3D float array, img, representing an RGB image in range [0.0,255.0]
    # Output: a 3D float array, img_pad, mirror padded so the number of rows and columns are multiples of 16
    
    M, N = img.shape[0:2]
    pad_r = ((16 - (M % 16)) % 16) # number of rows to pad
    pad_c = ((16 - (N % 16)) % 16) # number of columns to pad
    img_pad = np.pad(img, ((0,pad_r), (0,pad_c), (0,0)), "symmetric") # symmetric padding
    
    return img_pad
def encode_image(img, quality=75):
    # Inputs:  a 3D float array, img, representing an RGB image in range [0.0,255.0]
    #          an int, quality, the JPEG quality factor in range [1,100] (defaults to 75)
    # Outputs: a bitarray, Y_dc_bits, the Y component DC bitstream
    #          a bitarray, Y_ac_bits, the Y component AC bitstream
    #          a bitarray, Cb_dc_bits, the Cb component DC bitstream
    #          a bitarray, Cb_ac_bits, the Cb component AC bitstream
    #          a bitarray, Cr_dc_bits, the Cr component DC bitstream
    #          a bitarray, Cr_ac_bits, the Cr component AC bitstream
    
    M_orig, N_orig = img.shape[0:2]
    img = mirror_pad(img[:,:,0:3])
    M, N = img.shape[0:2]
    
    im_ycbcr = RGB2YCbCr(img)    
    Y = im_ycbcr[:,:,0]
    Cb = chroma_downsample(im_ycbcr[:,:,1])
    Cr = chroma_downsample(im_ycbcr[:,:,2])
    
    # Y component
    Y_dc_bits = bitarray.bitarray()
    Y_ac_bits = bitarray.bitarray()
    for i in np.r_[0:M:8]:
        for j in np.r_[0:N:8]:
            block = Y[i:i+8,j:j+8]
            dc_bits, ac_bits = encode_block(block, "y", quality)
            Y_dc_bits.extend(dc_bits)
            Y_ac_bits.extend(ac_bits)
    
    # Cb component
    Cb_dc_bits = bitarray.bitarray()
    Cb_ac_bits = bitarray.bitarray()
    for i in np.r_[0:M//2:8]:
        for j in np.r_[0:N//2:8]:
            block = Cb[i:i+8,j:j+8]
            dc_bits, ac_bits = encode_block(block, "c", quality)
            Cb_dc_bits.extend(dc_bits)
            Cb_ac_bits.extend(ac_bits)
            
    # Cr component
    Cr_dc_bits = bitarray.bitarray()
    Cr_ac_bits = bitarray.bitarray()
    for i in np.r_[0:M//2:8]:
        for j in np.r_[0:N//2:8]:
            block = Cr[i:i+8,j:j+8]
            dc_bits, ac_bits = encode_block(block, "c", quality)
            Cr_dc_bits.extend(dc_bits)
            Cr_ac_bits.extend(ac_bits)
            
    bits = (Y_dc_bits, Y_ac_bits, Cb_dc_bits, Cb_ac_bits, Cr_dc_bits, Cr_ac_bits)
    
    return bits

In [28]:
def encoder(img, outfile, quality=75,rate=12):
    # Inputs: a 3D uint8 array, img, representing an RGB color image
    #         a string, outfile, of the output binary filename
    #         an int, quality, the JPEG quality factor in range [1,100] (defaults to 75)
    
    O = len(img)
    M, N = img[0].shape[0:2]
    img = img[:,::3,::3,:]
    print('frames are:', O)
    img = img.astype(np.float64)
    
    bits = [] 
    quality = 40
    for i in range(O):
        g = encode_image(img[i], quality=quality)
        bits.append(g)
    print('len of bits:\n', len(bits))
    #return bits
    
    with open(outfile, "wb") as fh:
        # Your code here
        fh.write(bytes.fromhex("FFD1"))
        fh.write(M.to_bytes(2,'big'))
        fh.write(N.to_bytes(2,'big'))
        fh.write(quality.to_bytes(2,'big'))
        fh.write(rate.to_bytes(2,'big'))
        for a in bits:
            #print("encode:", a)
            fh.write(bytes.fromhex("FFD8"))
            for i in a:
                fh.write(bytes.fromhex("FFDA"))
                i.extend(bitarray.bitarray('1')*((16 - (len(i) % 16)) % 16))
                i.tofile(fh)
            fh.write(bytes.fromhex("FFD9"))
        fh.write(bytes.fromhex("FFD2"))
def compressed_to_b64(fname, compression,rate=12): 
    # compressed is a variable that contains the compressed data
    raw = imageStack_load(filename=fname)
    # raw = file we have compressed 
    outfile = "encoded.ww"
    ret_ = eval(compression)(raw,outfile=outfile,rate=rate) # should this be .bin file?
    with open(outfile, 'rb') as f:
        encoded = f.read()
    b64 = base64.b64encode(encoded)
    #print("\nb64", b64)
    return b64

In [14]:
A = imageStack_load(filename = 'photos_orig/Andy_Video.png')
print(A.shape)

(7, 168, 400, 3)


In [15]:
for i in range(len(A)):
    im_ = Image.fromarray(A[i])
    rec_fname = "photos_orig/Andy_Video/frame_{:02d}.tiff".format(i)
    im_.save(rec_fname,save_all=True)

In [16]:

GIF_save(path = 'photos_orig/Andy_Video/', framerate = 12)


In [29]:
#path = "photos_orig/Andy_Video/frame_{:02d}.tiff".format(1)
#a = Image.open(path)
#plt.imshow(a)
#Tiff_play(path= 'photos_orig/Andy_Video/*.tiff', display_size=400, frame_rate=12)
#GIF_save(path = 'photos_orig/Andy_Video/', framerate = 12)
fs = 48000
modem = TNCaprs(fs = fs ,Abuffer = 1024,Nchunks = 10)
callsign='KN6IES-5'
# For you to complete
fname = "photos_orig/dog.png"
address = 'NU6XB'
comment = 'github:video-compression'
print("Putting packets in Queue")
# For you to complete: Enqueue the file
#
data = compressed_to_b64(fname,compression="encoder",rate=12)
Qout, uid = enqueue_data(callsign=callsign, modem=modem, data=data, address=address, uid=None,
                         fname=fname, comment=comment, dest=b"APCAL", bsize=240)

print("Done. UID=%s" % uid.decode())
print("There are %d packets in the queue" %Qout.qsize())

Putting packets in Queue
frames are: 6
len of bits:
 6
Putting packets in Queue for transmission ID=1779
start: b':NU6XB    :START1779,photos_orig/dog.png,github:video-compression'
<class 'bytes'>
end: b':NU6XB    :END1779'
Done. UID=1779
There are 58 packets in the queue


In [21]:
# Send to our server
print("Starting transmission")
starttime = time.time()

while not(Qout.empty()):
    packet = Qout.get()
    #print(packet)
    time.sleep(2)
    sd.play(0.3*packet,samplerate=modem.fs,device=loop_idx,blocking=True)



print("Done")
print(time.time() - starttime)

Starting transmission
Done
235.18934297561646


In [1]:
from reconstruction import *

In [2]:
x = reconstruct("5529.bin","photos_orig/")

photos_orig/
b'\xff\xd1'
0 

reading bits
: bitarray('1110010000111001') 

0 

reading bits
: bitarray('11100100001110010000111001000011') 

0 

reading bits
: bitarray('111001000011100100001110010000111001000111100100') 

0 

reading bits
: bitarray('1110010000111001000011100100001110010001111001000111100110011100') 

0 

reading bits
: bitarray('11100100001110010000111001000011100100011110010001111001100111001000111001110011') 

0 

reading bits
: bitarray('111001000011100100001110010000111001000111100100011110011001110010001110011100110001111010100101') 

0 

reading bits
: bitarray('1110010000111001000011100100001110010001111001000111100110011100100011100111001100011110101001010010110001111100') 

0 

reading bits
: bitarray('11100100001110010000111001000011100100011110010001111001100111001000111001110011000111101010010100101100011111000001110000011100') 

0 

reading bits
: bitarray('11100100001110010000111001000011100100011110010001111001100111001000111001110011000111101010010100

this is run: 34
this is run: 35
this is run: 36
this is run: 37
this is run: 38
this is run: 39
this is run: 40
this is run: 41
this is run: 42
this is run: 43
this is run: 44
this is run: 45
this is run: 46
this is run: 47
this is run: 48
this is run: 49
this is run: 50
this is run: 51
this is run: 52
this is run: 53
this is run: 54
this is run: 55
this is run: 56
this is run: 57
this is run: 58
this is run: 59
this is run: 60
this is run: 61
this is run: 62
this is run: 63
this is run: 64
this is run: 65
this is run: 66
this is run: 67
this is run: 68
this is run: 69
this is run: 70
this is run: 71
this is run: 72
this is run: 73
this is run: 74
this is run: 75
this is run: 76
this is run: 77
this is run: 78
this is run: 79
this is run: 80
this is run: 81
this is run: 82
this is run: 83
this is run: 84
this is run: 85
this is run: 86
this is run: 87
this is run: 88
this is run: 89
this is run: 90
this is run: 91
this is run: 92
this is run: 93
this is run: 94
this is run: 95
this is 

KeyError: ('', 'Cannot find prefix in Huffman table.')